Doc2Vec을 반드시 텍스트 데이터에 이용해야 하는 것은 아닙니다. Sequential 한 아이템으로 서술할 수 있는 임의의 객체면 모두 Doc2Vec을 이용하여 임베딩 벡터를 학습할 수 있습니다. 

예를 들어 영화에는 배우가 casting order 순으로 기록되어 있습니다. 이 정보를 이용하여 캐스팅이 유사한 영화를 구할 수 있습니다

In [21]:
import pickle

idx2actor_file = 'data/navermovie_info_idx2actorname.pkl'
idx2movie_file = 'data/navermovie_info_idx2moviename.pkl'
document_file =  'data/navermovie_casting.txt'

with open(idx2actor_file, 'rb') as f:
    idx2actor = pickle.load(f)
    
with open(idx2movie_file, 'rb') as f:
    idx2movie = pickle.load(f)    
    
movie2idx = {name:idx for idx, name in idx2movie.items()}
actor2idx = {name:idx for idx, name in idx2actor.items()}    

In [22]:
movie2idx

{'너의 나라': '47968',
 '비만 블루스': '77317',
 '무브먼트 3': '131728',
 '어레스티드 디벨롭먼트': '63038',
 '불독': '81439',
 '인 블루': '133650',
 '부메랑': '34368',
 '브라더스 키스': '29394',
 '아메리칸 라이프': '120378',
 '고잉 투 타운': '56605',
 '시크릿 프렌드': '71580',
 '화소도': '11207',
 '이웃집 남자': '120278',
 '비셔스 독스': '95687',
 '간포채지뢰': '84800',
 '꼬마 공룡 딩크': '35161',
 '베리와 하루': '70760',
 '후회': '23773',
 '2000미터의 향수': '27461',
 '데이빗 코퍼필드': '35670',
 '스타-크로시드 러브': '89054',
 "켄트 나가노 슈만 교향곡 제3번 '라인'": '141015',
 '얼짱녀 어부': '114203',
 '미어 브라더 키 둘한': '109033',
 '오브 갓 앤 독스': '119704',
 '대지의 시인': '38973',
 '터미널 역': '71295',
 '비틀쥬스 - TV 만화 시리즈': '25882',
 '캠퍼스 S 커플': '117256',
 '더 지니어스 프롬 퀸티노': '98783',
 '노스 런던 북 오브 더 데드': '95742',
 '트루스 어바웃 케리': '137719',
 '마법의 눈': '12942',
 '사튜미 스토리 오브 하치로 레이크': '88682',
 '정의의 이름으로': '77005',
 '곰돌이 푸의 모험': '20315',
 '호세 리잘': '30928',
 '러브 앤 저스티스': '106939',
 '2대 두목은 크리스찬': '40628',
 '아메리카 방문객': '21879',
 '넌 어느 별에서 왔니': '46191',
 '나의 조각들': '112219',
 '마스터': '90590',
 '공포의 피라밋': '38754',
 '라라피포': '53287',
 '메

In [25]:
actor2idx

{'마릴리아 페라': '47968',
 '파멜린 치': '388227',
 '테리 콴': '131728',
 '알랑 드 카투엘란': '203690',
 '마리아 주앙 핀호': '332581',
 '김기정': '163220',
 '우마': '2248',
 '빅토리아 베라': '191039',
 '엘리자베스 리도': '333029',
 '메리 그로스': '29394',
 '스콧 피트': '210983',
 '템 홈즈': '120378',
 '크리스토퍼 에반 웰치': '56605',
 '샤펠 자페': '257745',
 '신동환': '43813',
 '클렘 소': '366394',
 '댄 맥캔': '318563',
 '매들린 새미': '100792',
 '헬레나 맥카시': '220686',
 '다도 제핸': '53145',
 '곽인진': '89375',
 'R.P. 에드워즈': '368971',
 'J. 애덤 브라운': '117593',
 '문다원': '146300',
 '험프리 커': '166948',
 '하비에르 피카요': '258131',
 '최은아': '89054',
 '임레 박사': '324832',
 '로버트 훅스': '20686',
 '헤일리 노엘 존슨': '8779',
 '카라 헤롤드': '5405',
 '앤드레 다이어츠': '209384',
 '곽민선': '374285',
 '린디 자밀 고메즈': '283813',
 '제이미 존스': '227687',
 '멜라니 밸레조': '159195',
 '존 밀러-몬존': '143090',
 '김의상': '41339',
 '엘비라 민구에즈': '166858',
 '배질 월리스': '70920',
 '스테파니 제이콥슨': '162399',
 '도널드 헤이든': '98783',
 '웬디 호프만': '259262',
 '레이첼 브로스나한': '159965',
 '바바라 코스타': '324296',
 '앤 울프': '186727',
 '키모 타빌라': '19668',
 '윌리엄 조셉 허친스': '273027',
 '론

영화 아이디를 document tags로, 배우를 단어로 취급합니다. Bag of Words와 비슷하게 Bag of Actors라는 이름의 클래스를 만들었습니다. 

    words = actors.split()
    
으로 만들어서 TaggedDocument의 words로 입력합니다

In [26]:
from gensim.models.doc2vec import TaggedDocument

class BagOfActors:
    def __init__(self, fname):
        self.fname = fname
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for row in f:
                movie_idx, actors = row.replace('\n','').split('\t')
                words = actors.split()
                if len(words) <= 1:
                    continue
                yield TaggedDocument(words = words, tags = ['MOVIE_ID = %s' % movie_idx])
        
        
bag_of_actors = BagOfActors(document_file)

for num, doc in enumerate(bag_of_actors):
    print(doc, '\n')
    if num > 3: break

TaggedDocument(['7995', '4009'], ['MOVIE_ID = 22682']) 

TaggedDocument(['154441', '12571'], ['MOVIE_ID = 25882']) 

TaggedDocument(['6852', '96763', '3107', '7067', '39104'], ['MOVIE_ID = 29394']) 

TaggedDocument(['187973', '339348'], ['MOVIE_ID = 120378']) 

TaggedDocument(['20842', '46829', '242523'], ['MOVIE_ID = 56605']) 



영화 배우는 한 번만 등장할 수도 있기 때문에 min_count=1로 하여 한 번이라도 등장한 배우는 이용하기로 합니다. 

size는 영화/배우가 임베딩되는 공간의 차원 크기입니다. 

In [27]:
from pprint import pprint
from gensim.models import Doc2Vec

doc2vec_model = Doc2Vec(bag_of_actors, size=50, min_count=1)

In [28]:
print(type(doc2vec_model.docvecs.doctags))
pprint(list(doc2vec_model.docvecs.doctags.items())[:3])

<class 'dict'>
[('MOVIE_ID = 22682', Doctag(offset=0, word_count=2, doc_count=1)),
 ('MOVIE_ID = 25882', Doctag(offset=1, word_count=2, doc_count=1)),
 ('MOVIE_ID = 29394', Doctag(offset=2, word_count=5, doc_count=1))]


영화가 임베딩이 되었고, 비슷한 영화를 검색할 수 있는지 확인해보았습니다. 이제 영화 아이디를 이름으로 치환하여 비슷한 영화들이 무엇인지를 확인해봅시다

In [29]:
doc2vec_model.docvecs.most_similar(1, topn=3)

[('MOVIE_ID = 138231', 0.5728163719177246),
 ('MOVIE_ID = 90033', 0.5624664425849915),
 ('MOVIE_ID = 22425', 0.5554822087287903)]

In [30]:
def movie_id2name(similar):
    idx = similar[0].split(' = ')[1]
    return (idx2movie.get(idx, 'unknown'), similar[1])

In [31]:
for similar in doc2vec_model.docvecs.most_similar(1000, topn=3):
    print(movie_id2name(similar))

('할렘 슈어', 0.5543142557144165)
('착신아리', 0.5504226684570312)
('먹고 자는 두사람 함께 사는 두사람', 0.5308488607406616)


송강호가 나왔던 영화 '관상'의 아이디를 찾기 위해서 movie2idx의 (key, value)를 for loop으로 돌면서 아이디를 찾았습니다. 영화 이름에 관상이 들어갔던 영화가 한 개 있었고, 아이디가 93728임을 확인합니다. 

In [32]:
for movie, idx in movie2idx.items():
    if '관상' in movie:
        print(movie, idx)

관상 93728


관상과 출연배우가 비슷한 영화를 찾았으나, 영화로는 잘 나오지 않았습니다. 사실 영화 자체가 배우 정보만 가지고 임베딩이 되기는 어렵습니다. 왜냐면 배우가 많아야 몇 십 편의 영화에 겨우 등장하며, 우리는 주연배우들에 의해서 영화 배우가 비슷하다고 판단하겠지만, 이 연습코드에서는 모든 배우의 중요도가 동일하였기 때문입니다. 

이 예시에서 한 가지 알 수 있는 것은, 임베딩을 학습하고 싶은 객체들을 잘 표현할 수 있는 데이터가 있어야 한다는 것입니다. 단순히 [영화 | 배우1, 배우2, ... ] 정보만으로는 임베딩이 어려울 수 있습니다

만약 주연 배우들의 힘을 더 실어주고 싶다면, 배우1이 4배, 배우2가 3배 더 중요하다면, 

    영화1 | 배우1 배우1 배우1 배우1 배우2 배우2 배우2 배우3 ...
    
와 같이 배우를 복제하고, Doc2Vec모델의 argument window를 크게 잡아줄 수 있습니다. 물론 임시방편인 방법입니다. 

In [33]:
for similar in doc2vec_model.docvecs.most_similar('MOVIE_ID = 93728', topn=30):
    print(movie_id2name(similar))

('캠프', 0.6116089820861816)
('야마모리 클립공장', 0.5371543169021606)
('팩 오브 마이 하트', 0.5320562720298767)
('더 빅 랜드', 0.5301306247711182)
('루스트', 0.5243034362792969)
('마법의 비디오 채널', 0.5236678719520569)
('크리스타니아의 전설', 0.5217958092689514)
('인스턴트 크레딧', 0.5112285614013672)
('콤비 걸', 0.5105454325675964)
('미란다 시즌1', 0.5033180117607117)
('원스 어폰 어 크라임', 0.5010089874267578)
('캡틴 론', 0.49658137559890747)
('아빠의 작은 조력자', 0.49274832010269165)
('화개반하', 0.4911739230155945)
('알레프랑스', 0.49081677198410034)
('라틴 러버스', 0.4887562394142151)
('내 여자친구는 구미호', 0.48689430952072144)
('고 다운 데스', 0.48091793060302734)
('프리쳐스 와이프', 0.4782719612121582)
('오프 밸런스', 0.47663724422454834)
('모디 투 더 맥스', 0.4765613377094269)
('지옥이 뭐가 나빠', 0.473810076713562)
('피칸☆치', 0.4729387164115906)
('101번째 프로포즈', 0.4699980616569519)
('테스', 0.4686300754547119)
('데스 아티스트, Inc.', 0.468394011259079)
('신편집부고사', 0.46771442890167236)
('희생양', 0.46748316287994385)
('쌍성계 : 고대 전설의 부활', 0.4665531814098358)
('보이프렌드 래트', 0.4633166193962097)


배우는 혹시 임베딩이 잘 되었는지 살펴보겠습니다

In [34]:
actor_names = ['송강호', '톰 크루즈', '안성기', '이선균', '장국영', '장쯔이']
for actor, idx in actor2idx.items():
    for actor_name in actor_names:
        if actor_name in actor:
            print(actor_name, idx)

장쯔이 2372
안성기 1843
장국영 1945
이선균 8017
송강호 1824
톰 크루즈 1558


In [35]:
doc2vec_model.most_similar('1843')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('7249', 0.9741384983062744),
 ('3191', 0.9735051393508911),
 ('8067', 0.9731763601303101),
 ('2808', 0.972918689250946),
 ('2806', 0.9729052186012268),
 ('6688', 0.9729011058807373),
 ('2011', 0.9728715419769287),
 ('11610', 0.9726345539093018),
 ('40869', 0.9725592136383057),
 ('2024', 0.9725563526153564)]

In [36]:
def actor_id2name(similar):
    idx = similar[0]
    return (idx2actor.get(idx, 'unknown'), similar[1])

In [37]:
print('탐 크루즈\n')
for similar in doc2vec_model.most_similar('1558'):
    print(actor_id2name(similar))

탐 크루즈

('바네사 레드그레이브', 0.6105495691299438)
('루파 매 퀸토', 0.6097619533538818)
('로렌 스캐처', 0.5752285718917847)
('오연안', 0.5645396113395691)
('길 모힌데폴 싱', 0.551176905632019)
('정천묵', 0.5507173538208008)
('미하엘 퓌스', 0.5492593050003052)
('로이드 하네스', 0.5489187240600586)
('칼라 구기노', 0.5460127592086792)
('해쉬 파텔', 0.5432533025741577)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


탐크루즈와 다르게 송강호는 한국배우들이 비슷한 출연을 했음을 확인할 수 있습니다

In [38]:
print('송강호\n')
for similar in doc2vec_model.most_similar('1824'):
    print(actor_id2name(similar))

송강호

('안토니 바슬러', 0.6377205848693848)
('조셉 신트론', 0.5627352595329285)
('길달호', 0.5512840747833252)
('니콜라 포제너', 0.5446561574935913)
('이사벨라 애스터', 0.5432105660438538)
('아만다 위스', 0.5406107306480408)
('찬드라 웨스트', 0.5362998247146606)
('제임스 테일러', 0.5343101024627686)
('키요키바 슌스케', 0.5290367603302002)
('정염려', 0.5237590074539185)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


안성기 배우는 연기 생활의 경력이 많기 때문에 오래전에 활동하셨던 김무생, 유인촌, 신구 등의 배우와 유사성이 있음을 확인할 수 있습니다. 영화는 학습이 잘 되지 않았지만, 배우는 아주 조금은 학습이 되었음을 확인할 수 있습니다. 

In [39]:
print('안성기\n')
for similar in doc2vec_model.most_similar('1843'):
    print(actor_id2name(similar))

안성기

('김희갑', 0.9741384983062744)
('남궁원', 0.9735051393508911)
('이대엽', 0.9731763601303101)
('허장강', 0.972918689250946)
('황해', 0.9729052186012268)
('장동휘', 0.9729011058807373)
('최무룡', 0.9728715419769287)
('구봉서', 0.9726345539093018)
('최남현', 0.9725592136383057)
('하명중', 0.9725563526153564)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


젊은 배우 이선균을 검색하면 이러한 경향을 좀 더 확인할 수 있습니다

In [40]:
print('이선균\n')
for similar in doc2vec_model.most_similar('8017'):
    print(actor_id2name(similar))

이선균

('데이빗 호위', 0.6236898899078369)
('강희세', 0.6006741523742676)
('츠유카', 0.5836237668991089)
('이본느 마리차 고메즈', 0.5523310303688049)
('사라 코엔', 0.5497802495956421)
('임강성', 0.5471169352531433)
('랜디 조셀린', 0.543982207775116)
('윌 르 바우', 0.5421175956726074)
('알렉스 브레머', 0.5411850214004517)
('도린 케오그', 0.5408064723014832)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


홍콩 배우 장국영을 검색하니 다른 홍콩 배우들이 등장함을 볼 수 있습니다

In [41]:
print('장국영\n')
for similar in doc2vec_model.most_similar('1945'):
    print(actor_id2name(similar))
    

장국영

('수잔 키거', 0.60575270652771)
('조지 A. 지메네즈', 0.6026967763900757)
('시에라 사무엘', 0.5963883399963379)
('존 알타무라', 0.5680398941040039)
('김준형', 0.5485057830810547)
('자니 크리어', 0.5284551382064819)
('조시 레이몬 아르고이티아', 0.5269392728805542)
('박희도', 0.5242489576339722)
('존 그리핀', 0.5231708884239197)
('장학우', 0.5226054191589355)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [42]:
print('장쯔이\n')
for similar in doc2vec_model.most_similar('2372'):
    print(actor_id2name(similar))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


장쯔이

('아드리아나 다비도바', 0.6063317060470581)
('타미 페스카텔리', 0.5857136249542236)
('김일희', 0.5770112872123718)
('시바키 타케루', 0.5683913230895996)
('샤롯 그렉', 0.5651161670684814)
('최은영', 0.5573536157608032)
('제이슨 콜렛', 0.5445411801338196)
('베스 트롤란', 0.5412739515304565)
('매튜 데이빗 제임스', 0.5369059443473816)
('최복생', 0.5347604155540466)
